In [1]:
import pandas as pd
import numpy as np

In [11]:
rmangal_metadata_path = "../../data/networks/all_rmangal_networks_metadata.csv"
network_global_to_rmangal_id_path = "../../data/networks/all/networks_metadata.csv"
parsed_rmangal_metadata_path = "../../data/networks/parsed_rmangal_networks_metadata.csv"
rmangal_geo_data_path = "../../data/metadata/network_level/rmangal_networks_geodata.csv"

bioclimate_varnames = {"BioClim_1": "Annual Mean Temperature",
"BioClim_2": "Mean Diurnal Range (Mean of monthly (max temp - min temp))",
"BioClim_3": "Isothermality (BioClim_2/BioClim_7) (×100)",
"BioClim_4": "Temperature Seasonality (standard deviation ×100)",
"BioClim_5": "Max Temperature of Warmest Month",
"BioClim_6": "Min Temperature of Coldest Month",
"BioClim_7": "Temperature Annual Range (BioClim_5-BioClim_6)",
"BioClim_8": "Mean Temperature of Wettest Quarter",
"BioClim_9": "Mean Temperature of Driest Quarter",
"BioClim_10": "Mean Temperature of Warmest Quarter",
"BioClim_11": "Mean Temperature of Coldest Quarter",
"BioClim_12": "Annual Precipitation",
"BioClim_13": "Precipitation of Wettest Month",
"BioClim_14": "Precipitation of Driest Month",
"BioClim_15": "Precipitation Seasonality (Coefficient of Variation)",
"BioClim_16": "Precipitation of Wettest Quarter",
"BioClim_17": "Precipitation of Driest Quarter",
"BioClim_18": "Precipitation of Warmest Quarter",
"BioClim_19": "Precipitation of Coldest Quarter"}

In [24]:
rmangal_metadata = pd.read_csv(rmangal_metadata_path)
network_global_to_rmangal_id = pd.read_csv(network_global_to_rmangal_id_path)

In [25]:
rmangal_metadata.geom_lat = rmangal_metadata.geom_lat.apply(lambda x: np.mean([float(i.replace("c(","").replace(")","")) for i in x.split(",")]) if type(x) is str else x)
rmangal_metadata.geom_lon = rmangal_metadata.geom_lon.apply(lambda x: np.mean([float(i.replace("c(","").replace(")","")) for i in x.split(",")]) if type(x) is str else x)

In [27]:
rmangal_metadata = rmangal_metadata.dropna(subset=["geom_lat", "geom_lon"]).rename(columns={"network_id": "ID",
                                                                                            "geom_lat": "Latitude",
                                                                                            "geom_lon": "Longitude"})

In [34]:
network_global_to_rmangal_id = network_global_to_rmangal_id.loc[network_global_to_rmangal_id.orig_path.str.contains("mangal")]
network_global_to_rmangal_id["rmangal_id"] = network_global_to_rmangal_id.orig_path.apply(lambda p: int(p.split("/")[-1].replace(".csv","")))

In [36]:
rmangal_metadata = rmangal_metadata.merge(network_global_to_rmangal_id[["network_index","rmangal_id"]], left_on="ID", right_on="rmangal_id").drop(["rmangal_id"], axis=1)

In [44]:
rmangal_metadata.to_csv(parsed_rmangal_metadata_path)

In [15]:
rmangal_metadata = pd.read_csv(parsed_rmangal_metadata_path)
rmangal_geodata = pd.read_csv(rmangal_geo_data_path).rename(columns=bioclimate_varnames).drop(["EcoRegion.y"], axis=1).rename(columns={"EcoRegion.x":"EcoRegion"})

In [17]:
rmangal_metadata = rmangal_metadata.merge(rmangal_geodata, left_on="ID", right_on="Obs_ID", how="left")

In [19]:
rmangal_metadata.to_csv(parsed_rmangal_metadata_path)